In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os

In [2]:
def process_swat_data(raw_file, output_file, is_attack=False):
    """
    處理 SWaT 原始數據並保存為 CSV
    """
    try:
        # 根據是否為攻擊數據選擇不同的讀取方式
        if is_attack:
            # 攻擊數據只有一行標題
            df = pd.read_csv(raw_file)
        else:
            # 正常數據有兩行標題，使用第二行作為列名
            df = pd.read_csv(raw_file, skiprows=1)

        print(f"\n讀取的資料形狀: {df.shape}")
        print("列名:", df.columns.tolist())
        
        # 確保時間戳列名正確
        timestamp_col = df.columns[0]
        df = df.rename(columns={timestamp_col: 'Timestamp'})
        
        # 清理時間戳數據
        df['Timestamp'] = df['Timestamp'].astype(str).str.strip()
        print("\n時間戳示例:")
        print(df['Timestamp'].head())
        
        # 轉換時間戳
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %I:%M:%S %p')
        
        # 將所有其他列轉換為數值型
        for col in df.columns:
            if col != 'Timestamp' and col != 'Normal/Attack':
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # 處理缺失值
        df = df.fillna(method='ffill').fillna(method='bfill')
        
        # 標準化數值列
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            scaler = MinMaxScaler()
            df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
        
        # 保存處理後的數據
        df.to_csv(output_file, index=False)
        print(f"\n已保存處理後的資料到 {output_file}")
        print(f"最終資料形狀: {df.shape}")
        
        return df
        
    except Exception as e:
        print(f"\n處理文件時出錯 {raw_file}:")
        print(f"錯誤詳情: {str(e)}")
        if 'df' in locals():
            print("\n資料資訊:")
            print(df.info())
        raise

In [3]:
def prepare_swat_dataset(normal_raw, attack_raw, output_dir):
    """
    準備完整的 SWaT 數據集
    """
    try:
        os.makedirs(output_dir, exist_ok=True)
        
        print("\n處理正常運行資料...")
        normal_df = process_swat_data(
            normal_raw, 
            os.path.join(output_dir, 'SWaT_normal.csv'),
            is_attack=False
        )
        
        print("\n處理攻擊資料...")
        attack_df = process_swat_data(
            attack_raw, 
            os.path.join(output_dir, 'SWaT_attack.csv'),
            is_attack=True
        )
        
        print("\n資料集準備完成!")
        print(f"正常資料形狀: {normal_df.shape}")
        print(f"攻擊資料形狀: {attack_df.shape}")
        
        return normal_df, attack_df
        
    except Exception as e:
        print(f"\n準備數據集時出錯: {str(e)}")
        raise

In [4]:
if __name__ == "__main__":
    normal_df, attack_df = prepare_swat_dataset(
        normal_raw='SWaT_Dataset_Normal_v1.csv',
        attack_raw='SWaT_Dataset_Attack_v0.csv',
        output_dir='processed_data'
    )


處理正常運行資料...

讀取的資料形狀: (495000, 53)
列名: [' Timestamp', 'FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'AIT201', 'AIT202', 'AIT203', 'FIT201', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', 'MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603', 'Normal/Attack']

時間戳示例:
0    22/12/2015 4:30:00 PM
1    22/12/2015 4:30:01 PM
2    22/12/2015 4:30:02 PM
3    22/12/2015 4:30:03 PM
4    22/12/2015 4:30:04 PM
Name: Timestamp, dtype: object


C:\Users\Dinosaur\AppData\Local\Temp\ipykernel_12036\2645419572.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')



已保存處理後的資料到 processed_data\normal.csv
最終資料形狀: (495000, 53)

處理攻擊資料...

讀取的資料形狀: (449919, 53)
列名: [' Timestamp', 'FIT101', 'LIT101', ' MV101', 'P101', 'P102', ' AIT201', 'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201', ' P202', 'P203', ' P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301', 'MV302', ' MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602', 'P603', 'Normal/Attack']

時間戳示例:
0    28/12/2015 10:00:00 AM
1    28/12/2015 10:00:01 AM
2    28/12/2015 10:00:02 AM
3    28/12/2015 10:00:03 AM
4    28/12/2015 10:00:04 AM
Name: Timestamp, dtype: object


C:\Users\Dinosaur\AppData\Local\Temp\ipykernel_12036\2645419572.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')



已保存處理後的資料到 processed_data\attack.csv
最終資料形狀: (449919, 53)

資料集準備完成!
正常資料形狀: (495000, 53)
攻擊資料形狀: (449919, 53)
